In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import (
    Dense, Dropout, GlobalAveragePooling2D,
    BatchNormalization, Input
)
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def load_dataset(train_dir, val_dir, test_dir, batch_size=32, img_size=(300, 300)):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.9, 1.1],
        fill_mode="nearest"
    )

    val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_ds = train_datagen.flow_from_directory(
        train_dir, target_size=img_size,
        batch_size=batch_size, class_mode="binary"
    )

    val_ds = val_test_datagen.flow_from_directory(
        val_dir, target_size=img_size,
        batch_size=batch_size, class_mode="binary"
    )

    test_ds = val_test_datagen.flow_from_directory(
        test_dir, target_size=img_size,
        batch_size=batch_size, class_mode="binary"
    )

    return train_ds, val_ds, test_ds

def build_model(input_shape=(300, 300, 3)):
    inputs = Input(shape=input_shape)
    
    base_model = InceptionResNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs
    )
    
    # Freeze base model first
    base_model.trainable = False
    
    # Unfreeze last 150 layers for fine-tuning
    for layer in base_model.layers[-150:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Reduced regularization
    x = Dense(1024, activation='swish', kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(512, activation='swish', kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(256, activation='swish')(x)
    x = BatchNormalization()(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    return model

# Dataset paths
train_dir = "/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Train"
val_dir = "/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation"
test_dir = "/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test"

batch_size = 32
img_size = (300, 300)

train_ds, val_ds, test_ds = load_dataset(train_dir, val_dir, test_dir, batch_size, img_size)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)

model = build_model()
optimizer = AdamW(learning_rate=0.0001, weight_decay=0.0001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluation
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Save model
model.save('/kaggle/working/model1.keras')

# Visualization functions remain the same
plot_training_curves(history)
plot_confusion_matrix(model, test_ds)

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

model = load_model('/kaggle/working/model1.keras')

inference_dir = "/path/to/your/inference/images"

img_size = (300, 300)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

predictions = []

for img_name in os.listdir(inference_dir):
    img_path = os.path.join(inference_dir, img_name)
    
    img_array = preprocess_image(img_path)
    
    prediction = model.predict(img_array)
    predictions.append((img_name, prediction[0][0]))
    
for img_name, pred in predictions:
    print(f"Image: {img_name}, Prediction: {'Faulty' if pred > 0.5 else 'Normal'} (Score: {pred:.4f})")